# Prepare Session for Batch Processing

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
#from vso import util, data
import astropy.units as u

Set parameters: object and tag name, image and session roots

In [ ]:
import vso.util
import vso.data

OBJ_NAME='SA38'
SESSION_TAG='20240731'
IMAGE_ROOT = '/srv/public/img'
WORK_ROOT = '/srv/public'

session = vso.util.Session(tag=SESSION_TAG, name=OBJ_NAME)
layout = vso.util.WorkLayout(WORK_ROOT)
session_layout = layout.get_session(session)
IMAGE_DIR = vso.util.ImageLayout(IMAGE_ROOT).get_images(session).lights_dir
sd = vso.data.StarData(layout.charts_dir)

Preview images

In [ ]:
from SetupSession import ImagePreview

preview = ImagePreview(IMAGE_DIR, sd, session_layout.solved_dir)
preview.run()


Adjust photometric apertures

In [ ]:
from photutils.aperture import (ApertureStats,
                                CircularAperture,
                                CircularAnnulus)
from astropy.nddata import Cutout2D

TILE_SIZE=40

size=(TILE_SIZE, TILE_SIZE)

image = preview.image
cutouts =  list([Cutout2D(image.data,
                    pos,
                    size,
                    image.wcs) for pos in preview.chart['radec2000']])





In [ ]:

pix_coord = image.wcs.world_to_pixel(preview.chart['radec2000'])
apr = CircularAperture(zip(*pix_coord), r=10)
ap_stats = ApertureStats(image, apr)


In [ ]:
from contextlib import contextmanager
import matplotlib.pyplot as plt
from astropy.visualization import AsinhStretch, ImageNormalize, MinMaxInterval
from matplotlib.patches import Circle
import ipywidgets as widgets
from IPython.display import display


def iterate_grid(items, grid, fig=None):
    for item, n in zip(items, range(len(items))):
        row = n // grid.ncols
        col = n % grid.ncols
        if fig is None:
            yield (item, grid[row, col])
        else:
            yield (item, fig.add_subplot(grid[row, col]))

@contextmanager
def grid_figure(items, ncols=4, width=6.40):
    nrows = (len(items)+1) // ncols + 1
    fig = plt.figure(figsize=(width, nrows * width / ncols))
    gs = fig.add_gridspec(nrows, ncols)
    try:
        yield fig, iterate_grid(items, gs)
    finally:
        plt.tight_layout()
        plt.show()

def default_image_normalizer(data):
    interval = MinMaxInterval()
    vmin, vmax = interval.get_limits(data)
    return ImageNormalize(vmin=vmin, vmax=vmax, stretch=AsinhStretch())


def plot_cutout(fig, cutout, position,
                subplot=None,
                norm=default_image_normalizer):

    ax = plt.subplot(projection=cutout.wcs) if subplot is None else fig.add_subplot(subplot, projection=cutout.wcs)
    ax.imshow(cutout.data, origin='lower', norm=norm(cutout.data))
    ax.coords[0].set_axislabel(position.ra.to_string(u.hour))
    ax.coords[1].set_axislabel("")
    ax.coords[0].set_ticks([] * u.degree)
    ax.coords[1].set_ticks([] * u.degree)
    return ax

def circle(ax, center, radius, **kwargs):
    ax.add_patch(Circle(center, radius=radius, **kwargs))

NUM_COL=6

wgt_enable = [widgets.Checkbox(value=True, description=star['auid'], indent=False) for star in preview.chart]
stars_disabled = set()
def cb_updated(event):
    owner = event['owner']
    star = owner.description
    if owner.value:
        stars_disabled.remove(star)
    else:
        stars_disabled.add(star)

for wgt in wgt_enable:
    wgt.observe(cb_updated, 'value')

wgt_grid = widgets.GridBox(wgt_enable, layout=widgets.Layout(
    grid_template_columns=f"repeat({NUM_COL}, {1/NUM_COL:.2%})"))
#    grid_template_columns=f"repeat({NUM_COL}, auto", width='90%'))

def plot_apertures(r_ap, r_ann):
    display(wgt_grid)
    r_in, r_out = r_ann
    with grid_figure(list(zip(cutouts, preview.chart, ap_stats.sky_centroid)),
                    ncols=NUM_COL, width=10.24) as (fig, cells):
        fig.suptitle(f"{image.header['object']}, {preview.band_}, "
                        f"UT {image.header['date-obs'].replace('T', ' ')}")
        for (cutout, star, centroid), cell in cells:
            ax = plot_cutout(fig, cutout, star['radec2000'], subplot=cell)
            ax.set_title(star['auid'])
            c = cutout.wcs.world_to_pixel(centroid)
            circle(ax, c, r_ap,
                linewidth=2, edgecolor='orange', facecolor='none', alpha=.5)
            circle(ax, c, r_in,
                linewidth=2, edgecolor='orange', facecolor='none', alpha=.5)
            circle(ax, c, r_out,
                linewidth=2, edgecolor='orange', facecolor='none', alpha=.5)
            if star['auid'] in stars_disabled:
                ax.plot([0, TILE_SIZE], [0, TILE_SIZE], color='red')
                ax.plot([TILE_SIZE, 0], [0, TILE_SIZE], color='red')

wgt_ap = widgets.FloatSlider(value=5, min=1, max=10, step=.1, desciption="Aperture")
wgt_ann = widgets.FloatRangeSlider(value=(10, 15), min=1, max=40, step=.1, desciption="Annulus")

def ap_updated(*args):
    if wgt_ann.value[0] < wgt_ap.value:
        d_ann = wgt_ann.value[1] - wgt_ann.value[0]
        wgt_ann.value = (wgt_ap.value + 1, wgt_ap.value + 1  + d_ann)
    wgt_ann.min  = wgt_ap.value

wgt_ap.observe(ap_updated, 'value')

widgets.interact_manual(plot_apertures,
                r_ap=wgt_ap,
                r_ann=wgt_ann)
